In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Read in the top 100 movies by gross from Box Office Mojo

In [ ]:
import requests
from bs4 import BeautifulSoup



url = 'http://www.boxofficemojo.com/yearly/chart/?yr=1983&p=.htm'

response = requests.get(url)

if response.ok:
    page = response.text
    soup = BeautifulSoup(page)

    tables = soup.find_all('table') # top level table but whole page is one big table
    nested_table = tables[3].find_all('table') # find all the sub tables used for layout
    
    d={}
    for row in nested_table[2].find_all('tr')[2:]:
        temp=[]
        for cell in row.find_all('td'):
            temp.append(cell.text)
        print(temp)
        d[temp[1]] = temp[2:]
        
    

__Once the movies have been downloaded, hit up wikipedia to see if there is a soundtrack for each movie. This is a little hit or miss.__ 

In [ ]:
# https://en.wikipedia.org/wiki/The_Hunger_Games:_Catching_Fire_–_Original_Motion_Picture_Soundtrack
# https://en.wikipedia.org/wiki/Frozen_(soundtrack)
import time
import random as random

http_string = "https://en.wikipedia.org/wiki/"
soundtrack_suffix1 = "_(soundtrack)"
soundtrack_suffix2 = "_–_Original_Motion_Picture_Soundtrack"
soundtrack_suffix3 = ":_Music_from_the_Motion_Picture"
soundtrack_suffix4 = "_(album)"

url_dict = {}

for title in d.keys():
    url = ""
    wikiUrl = http_string + title.replace(" ","_") + soundtrack_suffix2
    timetosleep = random.randrange(2)
    print(wikiUrl)
    wikiresponse = requests.get(wikiUrl)
    print(wikiresponse)
    if wikiresponse.ok:
        url = wikiUrl
        
    else:
        wikiUrl2 = http_string + title.replace(" ","_") + soundtrack_suffix1
        time.sleep(timetosleep/2)
        wikiresponse2 = requests.get(wikiUrl2)
        print(wikiUrl2)
        print(wikiresponse2)
        if wikiresponse2.ok:
            url = wikiUrl2
        else:
            wikiUrl3 = http_string + title.replace(" ","_") + soundtrack_suffix3
            time.sleep(timetosleep/3)
            wikiresponse3 = requests.get(wikiUrl3)
            print(wikiUrl3)
            print(wikiresponse3)
            if wikiresponse3.ok:
                url = wikiUrl3 
    if url != '':
        url_dict[title] = url
    time.sleep(timetosleep)


__Take the time to save the URLS for each movie soundtrack and the domestic gross values to a csv to save for later.__

In [ ]:
import csv

with open('1983WikiUrls.csv', 'w') as csvfile:
    fieldnames = ['title', 'url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in url_dict.items():
        #print(row)
        writer.writerow({'title': row[0], 'url' : row[1]})

In [ ]:
#Rank Movie Title Studio Total Gross  Theaters Opening  Theaters Open Close

with open('1983DomesticGross.csv','w') as csvfile:
    #fieldnames=['Movie Title','Studio','Total Gross','Total Theaters','Opening Gross','Opening Theaters','Open Date','Close Date']
    fieldnames=['Movie Title','Studio','Total Gross','Total Theaters','Opening Gross','Opening Theaters','Open Date']
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in d.items():
        #print(row)
        #['Sony', '$101,470,202', '3,055', '$20,719,162', '3,055', '6/12', '10/6']
        #writer.writerow({'Movie Title': row[0], 'Studio' : row[1][0], 'Total Gross' : row[1][1], 'Total Theaters' : row[1][2],'Opening Gross' : row[1][3], 'Opening Theaters': row[1][4], 'Open Date': row[1][5], 'Close Date': row[1][6]}) 
        writer.writerow({'Movie Title': row[0], 'Studio' : row[1][0], 'Total Gross' : row[1][1], 'Total Theaters' : row[1][2],'Opening Gross' : row[1][3], 'Opening Theaters': row[1][4], 'Open Date': row[1][5]})   

__Read wiki pages to determine if movie had a single__ This is a little closer to valid to capturing single URLs if they are in the box on the right side of the page but if it is in the text, it won't pick it up. Outputs the exceptions to the screen. Also looks for the word single in the text of the page to determine if there is a single not in the table.

In [902]:
urlfiles = [str(x)+'WikiUrls.csv' for x in range(1980,2013)]
print(urlfiles)



['1980WikiUrls.csv', '1981WikiUrls.csv', '1982WikiUrls.csv', '1983WikiUrls.csv', '1984WikiUrls.csv', '1985WikiUrls.csv', '1986WikiUrls.csv', '1987WikiUrls.csv', '1988WikiUrls.csv', '1989WikiUrls.csv', '1990WikiUrls.csv', '1991WikiUrls.csv', '1992WikiUrls.csv', '1993WikiUrls.csv', '1994WikiUrls.csv', '1995WikiUrls.csv', '1996WikiUrls.csv', '1997WikiUrls.csv', '1998WikiUrls.csv', '1999WikiUrls.csv', '2000WikiUrls.csv', '2001WikiUrls.csv', '2002WikiUrls.csv', '2003WikiUrls.csv', '2004WikiUrls.csv', '2005WikiUrls.csv', '2006WikiUrls.csv', '2007WikiUrls.csv', '2008WikiUrls.csv', '2009WikiUrls.csv', '2010WikiUrls.csv', '2011WikiUrls.csv', '2012WikiUrls.csv']


In [903]:
import csv
import sys

url_dict2 = {}


#urlfiles = ['1999WikiUrls.csv']

for filename in urlfiles:
    f = open(filename, 'rt')
    try:
        reader = csv.reader(f)
        for row in reader:
            url_dict2[row[0]] = row[1]
    finally:
        f.close()
    
    url_dict2.pop("title")

singlesDict = {}
exception_list = set()
for movie,url in url_dict2.items():
    
    if url == '':
        continue
    else:
        print(url)
    timetosleep = random.randrange(4)
    
    response = requests.get(url)
    
    if not response.ok:
        print("error code:",response)
        continue
    
    page = response.text
    soup = BeautifulSoup(page)
    
    # gets the side box that holds the singles information
    tables = soup.findAll("table", { "class" : "infobox vevent haudio" })
    if len(tables) == 0:
        print("No side box")
        paragraphs = soup.findAll("div", { "class" : "mw-content-ltr" })

        for p in paragraphs[0].findAll('p'):
            #print(p.text)
            match = re.search(r'(single)',p.text)
            if match:
                exception_list.add(url)
                print(match.group())
                print('found in text')
        
        continue
        
    for table in tables:
        
        singlesList = tables[0].findAll('ol')
        #assert len(singlesList)==1
        if len(singlesList) !=1:
            print("None or more than one ordered list within box")
            paragraphs = soup.findAll("div", { "class" : "mw-content-ltr" })

            for p in paragraphs[0].findAll('p'):
                #print(p.text)
                match = re.search(r'(single)',p.text)
                if match:
                    exception_list.add(url)
                    print(match.group())
                    print('found in text')
            continue
    
        s_list = []
        for item in singlesList[0].findAll('li'):
            href=item.find('a')
            if href:
                song_url = href.attrs['href']
            details = item.text.split("Released:")
            title = details[0].replace("\n","").replace("\"",'')
            if len(details) > 1:
                releaseDate = details[1]
            s_list.append((title, song_url, releaseDate))
        
        singlesDict[movie] = s_list

        #print(singlesDict)
    
        time.sleep(timetosleep)

https://en.wikipedia.org/wiki/Spirit:_Stallion_of_the_Cimarron_(soundtrack)


/Users/andrewvlahutin/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/andrewvlahutin/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


https://en.wikipedia.org/wiki/Cruel_Intentions_(soundtrack)
None or more than one ordered list within box
https://en.wikipedia.org/wiki/Toy_Story_2_(soundtrack)
https://en.wikipedia.org/wiki/Harry_Potter_and_the_Sorcerer's_Stone_(soundtrack)
None or more than one ordered list within box
https://en.wikipedia.org/wiki/The_Empire_Strikes_Back_(soundtrack)
None or more than one ordered list within box
single
found in text
single
found in text
None or more than one ordered list within box
single
found in text
single
found in text
https://en.wikipedia.org/wiki/Sense_and_Sensibility_(soundtrack)
None or more than one ordered list within box
https://en.wikipedia.org/wiki/Open_Season_(album)
No side box
https://en.wikipedia.org/wiki/Partners_(album)
No side box
https://en.wikipedia.org/wiki/Sherlock_Holmes:_A_Game_of_Shadows_–_Original_Motion_Picture_Soundtrack
None or more than one ordered list within box
https://en.wikipedia.org/wiki/Avalon_(soundtrack)
None or more than one ordered list with

In [1450]:
print(singlesDict)

{'Kick-Ass': [('Kick Ass (We Are Young)', '/wiki/Kick_Ass_(We_Are_Young)', ' 2 May 2010 (UK)')], 'Spirit: Stallion of the Cimarron': [('Here I Am', '/wiki/Here_I_Am_(Bryan_Adams_song)', ' June 11, 2002')], 'Mulan': [('Reflection', '/wiki/Reflection_(song)', ' June 2, 1998')], 'Toy Story 2': [('When She Loved Me', '/wiki/When_She_Loved_Me', ' November 24, 1999')], 'A Cinderella Story': [('Our Lips Are Sealed', '/wiki/Our_Lips_Are_Sealed#Hilary_and_Haylie_Duff_version', ' 2004')], 'Batman Returns': [('Face to Face', '/wiki/Face_to_Face_(Siouxsie_and_the_Banshees_song)', ' July 13, 1992')], 'Batman Forever': [('Hold Me, Thrill Me, Kiss Me, Kill Me', '/wiki/Hold_Me,_Thrill_Me,_Kiss_Me,_Kill_Me', ' April 5, 1995'), ('Kiss from a Rose', '/wiki/Kiss_from_a_Rose', ' June 6, 1995'), ('The Riddler', '/wiki/The_Riddler_(song)', ' October 24, 1995'), ('Where Are You Now?', '/wiki/Where_Are_You_Now%3F_(Brandy_song)', ' 1995'), ('Smash It Up', '/wiki/Smash_It_Up#Cover_versions', ' 1995'), ('Nobody L

In [ ]:
len(singlesDict)

In [ ]:
count = 0
for i in singlesDict.values():
    count += len(i)

print(count)

In [ ]:
print(len(exception_list))

Try to find the charts table on the page and pull the charts and ranks. This doesn't work extremely well as each chart is different per song. It outputs the exceptions. 

In [911]:
# Scrape for pages that have the 'Weekly Charts' table
song_d = {}
exception_list2 = set()
for songList in singlesDict.values():
    #print(songList)
    for song in songList:
        song_title = song[0]
        song_url_string = song[1]
        
        if song_url_string == '':
            print("no song url for:", song_title)
        
        #to handle if the link on the previous page is completely formed already
        if not 'https' in song_url_string:
            song_url = "https://en.wikipedia.org" + song[1] + '#Charts'
        else:
            song_url = song_url_string
            
        #print(song_url)
    
        response = requests.get(song_url)

        page = response.text
        soup = BeautifulSoup(page)
    

        charts_d = {}
        charts_headline = soup.findAll("span",{"class" : "mw-headline", "id" : "Weekly_charts"})
        
        if not charts_headline:
            charts_headline = soup.findAll("span",{"class" : "mw-headline", "id" : "Chart_performance"})
        
        if not charts_headline:
            charts_headline = soup.findAll("span",{"class" : "mw-headline", "id" : "Charts"})
        
        if not charts_headline:
            charts_headline = soup.findAll("span",{"class" : "mw-headline", "id" : "Chart_positions"})
        
        if not charts_headline:
            charts_headline = soup.findAll("span",{"class" : "mw-headline", "id" : "Charts_and_sales"})
    
        if not charts_headline:
            charts_headline = soup.findAll("span",{"class" : "mw-headline", "id" : "Chart_history"})
    
        if not charts_headline:
            charts_headline = soup.findAll("span",{"class" : "mw-headline", "id" : "Charts_and_certifications"})
        
        if charts_headline:
            charts_table = charts_headline[0].parent.find_next_sibling()
            #if not charts_table:
            #    charts_table = charts_headline[0].parent.find_next_sibling("table")
            
            
            
            #print(charts_table)
            if charts_table:
                
                #header row
                #header = charts_table.find('tr')
                #if header:
                #    charts = header.find_next_sibling('tr')
                
                #if not charts:
                #    charts = charts_table
                
                for tr in charts_table:
                    chart = ''
                    peakposition = ''
                    chart = tr.find_next('td')
                    #if not chart:
                    #        chart = tr.find_next('td')
                    if not chart:
                        continue
                    peakposition = chart.find_next('td')
                    #remove citation
                    chart_text = re.sub(r'(\[[0-9]+\])','',chart.text).strip()
                    esc_chart_text = str.replace(chart_text,r'.','',) # make U.S. = US
                    
                    #([\[0-9\]]+)
                    if not esc_chart_text:
                        print('our chart name is blank for ',song_url)
                        continue
                        
                    if len(esc_chart_text) > 100:
                        print("something is wrong; chart is too long", esc_chart_text, song_url)
                        continue
                        
                    try:
                        peakposition = int(peakposition.text)
                    except ValueError:
                        print("not a valid peak", peakposition, song_url)
                        continue    
                        
                    if isinstance(peakposition,int):
                        charts_d[esc_chart_text] = peakposition
                    else:
                        print('our peak position is invalid for',song_url)
            else:
                print("No weekly charts table", song_url)
            
            if len(charts_d) > 0:
                song_d[song_title] = charts_d    
        else:
            paragraphs = soup.findAll("div", { "class" : "mw-content-ltr" })

            for p in paragraphs[0].findAll('p'):
                #print(p.text)
                match = re.search(r'(Billboard)',p.text)
                if match:
                    exception_list2.add(song_url)
                    #print(match.group())
                    print('Billboard found in text',song_url)
                    continue
            print("cant get chart data; no chart data", song_url)
    #print(song_d)
    #break
    

/Users/andrewvlahutin/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/andrewvlahutin/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


our chart name is blank for  https://en.wikipedia.org/wiki/Here_I_Am_(Bryan_Adams_song)#Charts
cant get chart data; no chart data https://en.wikipedia.org/wiki/When_She_Loved_Me#Charts
not a valid peak <td>Australia<sup class="reference" id="cite_ref-10"><a href="#cite_note-10">[10]</a></sup></td> https://en.wikipedia.org/wiki/Our_Lips_Are_Sealed#Hilary_and_Haylie_Duff_version#Charts
our chart name is blank for  https://en.wikipedia.org/wiki/Face_to_Face_(Siouxsie_and_the_Banshees_song)#Charts
something is wrong; chart is too long Peak positions[edit]


Chart (1995)
Peak
position


Australian ARIA Singles Chart
1


Austrian Singles Chart
3


Belgian Singles Chart (Flanders)
15


Belgian Singles Chart (Wallonia)
4


Canadian RPM Singles Chart
3


Canadian RPM Alternative 30
1


Dutch Top 40
9


Finland (Suomen virallinen lista)
1


French SNEP Singles Chart
10


German Singles Chart
8


Irish Singles Chart
1


New Zealand RIANZ Singles Chart
1


Norwegian Singles Chart
1


Swedish Singl

In [1208]:
print(song_d)

{'One Night Only': {'13': 67}, 'How Deep Is Your Love': {'US Billboard Hot 100': 71}, 'Wind Beneath My Wings': {'US Hot Country Songs (Billboard)': 4}, 'Now or Never': {'Irish Singles Chart': 42, 'Australian Singles Chart': 62, 'Danish Singles Chart': 18, 'US Billboard Hot 100': 68, 'Canadian Hot 100': 72, 'Swiss Singles Chart': 64, 'UK Singles Chart': 41, 'US Billboard Pop 100': 41}, 'For You I Will': {'US Adult Contemporary (Billboard)': 3, 'Australia (ARIA)': 45, 'New Zealand (Recorded Music NZ)': 2, 'Germany (Official German Charts)': 80, 'Netherlands (Dutch Top 40)': 51, 'US Billboard Hot 100': 4, 'US Rhythmic Top 40 (Billboard)': 3, 'US Hot R&B/Hip-Hop Songs (Billboard)': 2, 'US Mainstream Top 40 (Billboard)': 6, 'UK Singles (Official Charts Company)': 27}, 'Have a Party': {'US Bubbling Under Hot 100 Singles (Billboard)': 5}, 'Cluck Cluck': {'84': 52, '11': 19, '52': 11}, 'Down to Earth': {'Canadian Singles Chart': 73, 'UK Singles Chart': 118}, 'Little Wonders': {'US Adult Contem

In [1247]:
print(len(song_d))
count = 0
for i in singlesDict.values():
    count += len(i)

print(count)

193
270


'2012': [('Time for Miracles',
   '/wiki/Time_for_Miracles',
   ' October 18, 2009')],

Manually scrapped movie -> song relationship

In [1464]:
extra_songs = {
'Indecent Proposal' : [('In All the Right Places', 'wiki/In_All_the_Right_Places', '24 May 1993' )],
'The Jazz Singer' : [('Love on the Rocks' , 'wiki/Love_on_the_Rocks_(Neil_Diamond_song)', '1980'),
('Hello Again' , 'wiki/Hello_Again_(Neil_Diamond_song)', '1981'),
('America', 'wiki/America_(Neil_Diamond_song)', '1981')],
'Baby Boy': [('Just a Baby Boy', 'wiki/Just_a_Baby_Boy', 'June 2011')],
'Hope Floats' : [('Chances Are', '', '1998'),('To Get Me to You', '', '1998')],
'Save the Last Dance': [('Crazy', 'wiki/Crazy_(K-Ci_and_JoJo_song)','March 2, 2001'),('You', '' , '')],
'Any Given Sunday' : [('Shut \'em Down', 'wiki/Shut_%27Em_Down_(LL_Cool_J_song)', 'February 1, 2000')],
'Kazaam': [('Wishes', '', ''), ('I\'ll Make Your Dreams Come True','','')], 
'Booty Call': [('Can We', '/wiki/Can_We', 'February 11, 1997')],
'Romeo Must Die': [('Try Again', 'wiki/Try_Again_(Aaliyah_song)', 'February 22, 2000'),
('Come Back in One Piece', 'wiki/Come_Back_in_One_Piece','June 6, 2000'),
('We At It Again', '','')],
'A Thin Line Between Love and Hate': [('Beware of My Crew','',''),
('A Thin Line Between Love and Hate','',''),
('Let\'s Stay Together','','')],
'The Wood': [('I Wanna Know','wiki/I_Wanna_Know','November 7, 1999'),
('Neck uv da Woods','',''),('Crave','',''),
('Think About You','','')],
'Alvin and the Chipmunks': [('Witch Doctor (2007 Version)','',''),
('The Chipmunk Song (Christmas Don\'t Be Late)','',''),('Bad Day','',''),('Funkytown','','')],
'Mermaids': [('The Shoop Shoop Song (It\'s in His Kiss)','wiki/The_Shoop_Shoop_Song_(It%27s_in_His_Kiss)','November 7, 1990')],
'White Men Can\'t Jump': [('White Men Can\'t Jump', '','')],
'Blade': [ ('Wrek Tha Discotek','','')],
'Training Day': [('#1','wiki/Number_1_(Nelly_song)','January 29, 2002'),('Put It on Me','','')],
'Higher Learning': [('Ask of You','',''),('Situation: Grimm','','')],
'Cradle 2 the Grave': [('X Gon\' Give It to Ya','wiki/X_Gon%27_Give_It_to_Ya','January 20, 2003')],
'Romeo + Juliet': [('Lovefool','','')],
'Django Unchained': [('Freedom','',''),('Django','','')],
'You Got Served': [('Badaboom','','')],
'Menace II Society': [('Streiht Up Menace','',''), ('Trigga Gots No Heart','','')],
'Down to Earth': [('Someone to Love You','',''),('Just Another Girl','','')],
'Beverly Hills Cop': [('Axel F','',''),('The Heat Is On','','')],
'Fast & Furious' :[('Blanco','','',)],
'How High': [('Part II','','December 4, 2001')],
'Bad Company': [('All Out of Love','','')],
'Brown Sugar': [('Love of My Life (An Ode to Hip-Hop)','','')],
'Barbershop 2: Back in Business': [('Not Today','',''),('I Can\'t Wait','',''),('Never','','')],
'Summer School' : [('Mind Over Matter','','')],
'Jason\'s Lyric': [('If You Think You\'re Lonely Now','','')],
'Friday': [ ('Keep Their Heads Ringin','','March 7, 1995') ],
'Wild Wild West' : [ ('Wild Wild West','','')],
'The Fast and the Furious': [ ('POV City Anthem','','') ],
'Pretty in Pink': [ ('Bring On the Dancing Horses','','14 November 1985'),('If You Leave','','21 April 1986')],
'Barbershop': [ ('Stingy','','') ],
'New Jack City': [ ('I\'m Dreaming','',''),('For the Love of Money/Living for the City','','')],
'The Woman in Red': [ ('I Just Called to Say I Love You','','') ],
'House Party 2': [ ('Ain\'t Gonna Hurt Nobody','',''),('House Party (I Don\'t Know What You Come to Do)','',''),('Yo Baby Yo','','')],
'Shrek 2': [ ('Accidentally in Love','','')],
'Against All Odds': [ ('Against All Odds (Take a Look at Me Now)','','')],
'Big Momma\'s House': [ ('Bounce with Me','',''), ('I\'ve Got to Have It','','')],
'CB4': [ ('Baby Be Mine','','') ],
'All About the Benjamins': [('Told Y\'all','','')],
'Coal Miner\'s Daughter': [ ('Blue Moon of Kentucky','','')],
'Reality Bites': [ ('Stay (I Missed You)','','')],
'Bad Boys': [ ('Never Find Someone Like You','',''),('Someone to Love','',''), ('Shy Guy','','')],
'Xanadu': [ ('Magic','','')],
'The Land Before Time': [ ('If We Hold on Together','','') ],
'Endless Love': [ ('Endless Love','','')],
'The Wild Life': [('The Wild Life','','')],
'Hardball': [('Hardball','','')],
'Set It Off':  [( 'Don\'t Let Go (Love)','',''), ('Days of Our Livez','','')],
'Addams Family Values': [('Addams Family (Whoomp!)','','') ],
'White Nights': [('Separate Lives','','') ],
'Deep Blue Sea': [('Deepest Bluest','','') ],
'The Chronicles of Narnia: The Voyage of the Dawn Treader' : [ ('There\'s a Place for Us','','') ],
'Poetic Justice' : [ ('Again','','')],
'O Brother, Where Art Thou?' : [('O Brother, Where Art Thou?','','')],
'Street Fighter' : [('Something Kinda Funky','','') ],
'Fled' : [('Touch Myself','','') ],
'Exit Wounds': [('No Sunshine','','') ],
'Colors' : [('Colors','','')],
'Pulp Fiction': [('Girl, You\'ll Be a Woman Soon','','')],
'Bringing Down the House' : [('Better Than the Rest','','') ],
'Supercop': [ ('What\'s Love Got to Do With It','','')],
'Deep Cover': [('Deep Cover','','') ],
'Juice' : [('Uptown Anthem','',''), ('Juice (Know the Ledge)', '',''),('Don\'t Be Afraid','',''),('Is It Good to You','','')],
'Casino Royale' : [('You Know My Name','','') ],
'How Stella Got Her Groove Back' : [('Luv Me, Luv Me','','') , ('Beautiful','','')],
'Nothing to Lose' : [('Not Tonight','',''), ('C U When U Get There','','')],
'House Party 3' : [('Butt Booty Naked','','') ],
'Grease 2' : [('Back to School Again','','')],
'Days of Thunder' : [('Show Me Heaven','','')],
'Like Mike' : [('Basketball','',''),('Take Ya Home','','')],
'Money Train': [('Top of the Stairs','','')],
'Step Up 2: The Streets': [('Low','','')],
'Eddie': [('Tell Me','',''),('It\'s All the Way Live (Now)','','')],
'The Longest Yard': [('Errtime','','')],
'Phenomenon' : [('Change the World','','') ],
'The Matrix Reloaded' : [('Sleeping Awake','','')],
'Soul Food' : [('I Care \'Bout You','',''),('What About Us?','',''),('We\'re Not Making Love No More','',''),('A Song for Mama','','')],
'High School High': [('Wu-Wear: The Garment Renaissance','','')],
'Love & Basketball' : [('Dance Tonight','','')],
'Where the Wild Things Are' : [('All Is Love','','')],
'Beverly Hills Cop III': [('The Right Kinda Lover','',''),('Luv 4 Dem Gangsta\’z','','')]
}

In [1470]:
#merge songs_dict and extra songs
print(len(singlesDict))
print(len(extra_songs))

singlesDict.update(extra_songs)

print(len(singlesDict))


121
86
207


(When You Gonna) Give It Up to Me': {'Australian Singles Chart': 17,
  'Austria (Ö3 Austria Top 40)': 30,
  'Belgium (Ultratop 50 Flanders)': 22,
  'Belgium (Ultratop 50 Wallonia)': 14,
  'Czech Singles Chart': 12,
  'Finnish Singles Chart': 4,
  'French Singles Chart': 25,
  'Germany (Official German Charts)': 26,
  'Irish Singles Chart': 47,
  'Slovakia (Rádio Top 100)': 32,
  'Switzerland (Schweizer Hitparade)': 5,
  'UK Singles Chart': 31,
  'US Billboard Hot 100': 3,
  'US Billboard Hot R&B/Hip-Hop Songs': 5,
  'US Billboard Pop 100': 6,
  'US Billboard Pop Songs': 8,
  'US Billboard Rap Songs': 1},

Manually scrapped song -> chart data based on exceptions

In [1462]:
extra_charts = {
'Love on the Rocks' : {'US Billboard Hot 100' : 2},
'Hello Again' : {'US Billboard Hot 100': 6},
'America' : {'US Billboard Hot 100' : 8},
'Just a Baby Boy': {'US Billboard Hot 100' : 90, 'US Hot R&B/Hip-Hop Songs (Billboard)':40}, 
'Chances Are': {'US Adult Contemporary (Billboard)' : 23},
'Crazy' : { 'US Billboard Hot 100' : 11, 'US Hot R&B/Hip-Hop Songs (Billboard)' : 63},
'Shut \'em Down': { 'US Hot R&B/Hip-Hop Songs (Billboard)' : 88,'US Hot Rap Singles' : 31},
'Wishes': {'US Billboard Hot 100' : 86, 'US Hot R&B/Hip-Hop Songs (Billboard)' : 56},
'I\'ll Make Your Dreams Come True': {'US Hot R&B/Hip-Hop Songs (Billboard)' : 64},
'Can We': {'US Billboard Hot 100' : 75, 'US Hot R&B/Hip-Hop Songs (Billboard)' : 31},
'Try Again': {'US Billboard Hot 100' : 1, 'US Hot R&B/Hip-Hop Songs (Billboard)' : 4},
'I Wanna Know':{'US Billboard Hot 100' : 4, 'US Hot R&B/Hip-Hop Songs (Billboard)'  : 2, 'US Adult Contemporary (Billboard)' : 21},
'Witch Doctor (2007 Version)': {'US Billboard Hot 100'  : 62,'US Hot Digital Songs' : 16},
'The Chipmunk Song (Christmas Don\'t Be Late)': { 'US Billboard Hot 100'  :  66, 'US Hot Digital Songs' : 28},
'Bad Day' : { 'US Billboard Hot 100' :  67, 'US Hot Digital Songs': 51},
'Funkytown': { 'US Billboard Hot 100'  :  86 },
'The Shoop Shoop Song (It\'s in His Kiss)': {'US Billboard Hot 100': 33, 'US Adult Contemporary (Billboard)' : 7},
'White Men Can\'t Jump': {'US Billboard Hot 100' : 90},
'Wrek Tha Discotek': {'US Dance/Club Play' : 42},
'#1' : {'US Billboard Hot 100' : 22, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 20},
'Ask of You' : {'US Billboard Hot 100' : 19, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 2},
'Situation: Grimm' : {'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 97},
'X Gon\' Give It to Ya': {'US Billboard Hot 100' : 60, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 23},
'Lovefool': {'US Billboard Hot 100' : 2, 'U.S. Billboard Adult Contemporary' : 23},
'Badaboom': {'US Billboard Hot 100' : 59},
'Someone to Love You':{'US Billboard Hot 100' : 49},
'Just Another Girl': {'US Billboard Hot 100' : 64, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 34},
'Axel F': {'US Billboard Hot 100' : 3},
'The Heat Is On': {'US Billboard Hot 100' : 2},
'Love of My Life (An Ode to Hip-Hop)': {'US Billboard Hot 100' : 9, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 1},
'Not Today': {'US Billboard Hot 100' : 41, 'US Hot R&B/Hip-Hop Songs (Billboard)' : 21},
'Mind Over Matter': {'US Dance/Club Play' : 10},
'If You Think You\'re Lonely Now': {'US Billboard Hot 100' : 17, 'US Hot R&B/Hip-Hop Songs (Billboard)' : 11},
'Keep Their Heads Ringin': {'US Billboard Hot 100' : 10, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 10},
'Wild Wild West': {'US Billboard Hot 100' : 1},
'If You Leave': {'US Billboard Hot 100' : 4},
'Stingy': {'US Billboard Hot 100' : 33, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 7},
'I\'m Dreaming': { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 1},
'For the Love of Money/Living for the City': { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 12},
'I Just Called to Say I Love You' : {'US Billboard Hot 100' : 1},
'Accidentally in Love': {'US Billboard Hot 100' : 39},
'Against All Odds (Take a Look at Me Now)': {'US Billboard Hot 100' : 1},
'Bounce with Me': {'US Billboard Hot 100' : 20, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 1},
'I\'ve Got to Have It': { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 67},
'Baby Be Mine':  { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 17},
'Stay (I Missed You)': {'US Billboard Hot 100' : 1},
'Never Find Someone Like You': { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 42},
'Someone to Love': {'US Billboard Hot 100' : 10, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 7},
'Shy Guy': {'US Billboard Hot 100' : 13, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 21},
'Magic': {'US Billboard Hot 100' : 1},
'If We Hold on Together' : {'US Adult Contemporary (Billboard)' : 23},
'Endless Love':  {'US Billboard Hot 100' : 1},
'The Wild Life': {'US Billboard Hot 100' : 70},
'Hardball': { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 77},
'Don\'t Let Go (Love)' : {'US Billboard Hot 100' : 2, 'US Hot R&B Singles (Billboard)' : 1},
'Days of Our Livez' : { 'U.S. Billboard Hot Rap Tracks' : 12},
'Separate Lives' :  { 'U.S. Billboard Hot Rap Tracks' : 1},
'There\'s a Place for Us' : {'US Country Digital Songs (Billboard)': 11},
'Again' : {'US Billboard Hot 100' : 1, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 7},
'O Brother, Where Art Thou?' : {'U.S. Billboard Hot Country Singles & Tracks' : 35},
'Something Kinda Funky' : {'Hot Rap Singles' : 39},
'Touch Myself' : {'US Billboard Hot 100' : 40, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks': 23},
'No Sunshine' : { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 67},
'Colors' : {'US Billboard Hot 100' : 70},
'Girl, You\'ll Be a Woman Soon' : {'US Billboard Hot 100' : 59},
'What\'s Love Got to Do With It': { 'US Billboard Hot 100' : 59,'U.S. Billboard Hot Rap Tracks' : 5},
'Deep Cover' : { 'U.S. Billboard Hot Rap Tracks' : 4},
'Uptown Anthem' : { 'U.S. Billboard Hot Rap Tracks' : 58},
'You Know My Name' : {'US Billboard Hot 100' : 79},
'Luv Me, Luv Me' : { 'US Billboard Hot 100' : 75},
'Not Tonight' :  {'US Billboard Hot 100' : 6, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 3},
'C U When U Get There' : {'US Billboard Hot 100' : 12, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 34},
'Back to School Again' : {'US Billboard Hot 100' : 71},
'Show Me Heaven': {'US Adult Contemporary (Billboard)' : 28},
'Basketball': { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 44},
'Take Ya Home': {'US Billboard Hot 100' : 72, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 21},
'Top of the Stairs': {'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 73},
'Low' : {'US Billboard Hot 100' : 1, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 9},
'Tell Me' : {'US Billboard Hot 100' : 18, 'Hot R&B/Hip-Hop Songs' : 5},
'It\'s All the Way Live (Now)' : {'US Billboard Hot 100' : 29, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 47},
'Errtime': {'US Billboard Hot 100' : 24, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 59},
'Change the World' : {'US Billboard Hot 100' : 5},
'I Care \'Bout You' : {'US Billboard Hot 100' : 23},
'What About Us?': {'US Billboard Hot 100' : 16, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 4},
'We\'re Not Making Love No More' : {'US Billboard Hot 100' : 13},
'A Song for Mama' : {'US Billboard Hot 100' : 7, 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 1},
'Wu-Wear: The Garment Renaissance': {'US Billboard Hot 100' : 60},
'Dance Tonight' : { 'U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks' : 5},
'The Right Kinda Lover' : {'US Billboard Hot 100' : 61},
'Luv 4 Dem Gangsta\'z' : {'U.S. Billboard Hot Rap Tracks' : 26}
}
              


In [1472]:
#merge extra_charts with song_d

print(len(song_d))
print(len(extra_charts))
      
song_d.update(extra_charts)
      
print(len(song_d))

193
90
283


In [1207]:
print(exception_list2)

{'https://en.wikipedia.org/wiki/Dancing_in_the_Sheets#Charts', 'https://en.wikipedia.org/wiki/Sympathy_for_the_Devil#Guns_N.27_Roses_version#Charts', 'https://en.wikipedia.org/wiki/I_Will_Get_There#Charts', 'https://en.wikipedia.org/wiki/Under_the_Boardwalk#Covers#Charts', 'https://en.wikipedia.org/wiki/Rock_and_Roll_All_Nite#Poison_version#Charts', 'https://en.wikipedia.org/wiki/Shame_(Evelyn_King_song)#Charts', 'https://en.wikipedia.org/wiki/I%27m_Free_(Heaven_Helps_the_Man)#Charts', 'https://en.wikipedia.org/wiki/Don%27t_Stop_the_Dance#Charts', 'https://en.wikipedia.org/wiki/At_the_Beginning#Charts', 'https://en.wikipedia.org/wiki/Rock_Ice_(song)#Charts', 'https://en.wikipedia.org/wiki/All_Time_High#Charts', 'https://en.wikipedia.org/wiki/Quicksilver_Lightning#Charts', 'https://en.wikipedia.org/wiki/Where_the_Dream_Takes_You#Charts', 'https://en.wikipedia.org/wiki/Turtle_Power#Charts'}


In [1209]:
print(len(exception_list2))

14


In [1538]:
songs_df[songs_df["Movie Title"] =='Titanic']

#songs_df[songs_df["Movie Title"] =='The Prince of Egypt']

,Movie Title,Song Title,Wiki URL,Single Release Date,ChartBill100,ChartHotDance,ChartHipHop,ChartOther
321,Titanic,My Heart Will Go On,/wiki/My_Heart_Will_Go_On,"December 8, 1997",False,False,False,False


More manually scrapped data to add

In [1235]:
exceptionDict = {"Dancing in the Sheets" : {"US Billboard Hot 100" : 17, 
"US Hot R&B/Hip-Hop Songs (Billboard)" : 18}, 
"Sympathy for the Devil" : { "US Billboard Hot 100" : 55},
"I Will Get There" : {"US Billboard Hot 100"  : 32} ,
"Shame": {"US Billboard Hot 100" : 28, "US Hot R&B/Hip-Hop Songs (Billboard)" : 12},
"I'm Free (Heaven Helps the Man)" : {"US Billboard Hot 100"  : 22},
"Don't Stop the Dance" : {"US Billboard Adult Contemporary" : 26},
"At the Beginning" : {"US Billboard Hot 100"  :  45, "US Billboard Adult Contemporary" : 2},
"All Time High" : {"US Billboard Hot 100" :  36, "US Billboard Adult Contemporary" : 1},
"Quicksilver Lightning" :  { "US Billboard Mainstream Rock Tracks" : 11},
"Turtle Power" : { "US Billboard Hot 100" : 13, "US Hot Rap Songs (Billboard)" : 2} }

In [1465]:
song_d

{'(When You Gonna) Give It Up to Me': {'Australian Singles Chart': 17,
  'Austria (Ö3 Austria Top 40)': 30,
  'Belgium (Ultratop 50 Flanders)': 22,
  'Belgium (Ultratop 50 Wallonia)': 14,
  'Czech Singles Chart': 12,
  'Finnish Singles Chart': 4,
  'French Singles Chart': 25,
  'Germany (Official German Charts)': 26,
  'Irish Singles Chart': 47,
  'Slovakia (Rádio Top 100)': 32,
  'Switzerland (Schweizer Hitparade)': 5,
  'UK Singles Chart': 31,
  'US Billboard Hot 100': 3,
  'US Billboard Hot R&B/Hip-Hop Songs': 5,
  'US Billboard Pop 100': 6,
  'US Billboard Pop Songs': 8,
  'US Billboard Rap Songs': 1},
 '8 Mile': {'1': 14, '14': 4, '2': 1, '3': 2, '4': 2, '9': 1},
 'A Hazy Shade of Winter': {'UK Singles Chart': 30,
  'US Billboard Hot 100': 13},
 'A Night to Remember': {'Australian ARIA Singles Chart': 96,
  'US Billboard Bubbling Under Hot 100 Singles': 8},
 'A View to a Kill': {'Australia': 6,
  'Austria': 1,
  'Brazil': 1,
  'Canada': 1,
  'France': 11,
  'Germany': 9,
  'Irelan

In [1238]:
song_d.update(exceptionDict)

In [1473]:
#songs_chart_df = pd.DataFrame(song_d)
#songs_chart_df = pd.DataFrame.from_dict(song_d, orient='columns')

song_titles = []

charts = []

for title, d in song_d.items():
    song_titles.append(title)
    charts.append(pd.DataFrame.from_dict(d, orient='index'))
    
songs_chart_df = pd.concat(charts, keys=song_titles)

In [1384]:
list(singlesDict.values())[:3]

[[('Kick Ass (We Are Young)',
   '/wiki/Kick_Ass_(We_Are_Young)',
   ' 2 May 2010 (UK)')],
 [('Here I Am', '/wiki/Here_I_Am_(Bryan_Adams_song)', ' June 11, 2002')],
 [('Reflection', '/wiki/Reflection_(song)', ' June 2, 1998')]]

In [1475]:
songs_chart_df2 = songs_chart_df.reset_index()

In [1625]:
songs_chart_df2["Chart"].value_counts()

US Billboard Hot 100                               167
Australia (ARIA)                                    37
UK Singles Chart                                    35
UK Singles (Official Charts Company)                33
U.S. Billboard Hot R&B/Hip-Hop Singles & Tracks     31
US Hot R&B/Hip-Hop Songs (Billboard)                27
Ireland (IRMA)                                      23
New Zealand (Recorded Music NZ)                     21
Germany (Official German Charts)                    21
Netherlands (Dutch Top 40)                          21
Austria (Ö3 Austria Top 40)                         20
US Adult Contemporary (Billboard)                   20
France (SNEP)                                       20
Sweden (Sverigetopplistan)                          19
Belgium (Ultratop 50 Flanders)                      18
Irish Singles Chart                                 18
Switzerland (Schweizer Hitparade)                   16
German Singles Chart                                16
US Hot Dan

Some data wasn't scrapped correctly and numbers are put in the chart value. Some manually updates are needed.

In [1622]:
songs_chart_df2.loc[1097:1097]["Chart"] = 'US Dance Club Songs (Billboard)'
#songs_chart_df2.loc[422:422]["Chart Rank"] = 61

/Users/andrewvlahutin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [1610]:
songs_chart_df2[songs_chart_df2["Song Title"]=='Safe & Sound']

,Song Title,Chart,Chart Rank
100,Safe & Sound,42,67
101,Safe & Sound,38,62
102,Safe & Sound,62,31
103,Safe & Sound,31,11
104,Safe & Sound,44,42
105,Safe & Sound,11,44
106,Safe & Sound,67,30


In [1624]:
songs_chart_df2.rename(columns={'level_0': "Song Title", 'level_1': 'Chart', 0 : 'Chart Rank' }, inplace = True)
#My Heart Will Go On
songs_chart_df2[songs_chart_df2["Chart"]=='71']

,Song Title,Chart,Chart Rank
95,Make a Movie,71,6


In [1626]:
#s.str.startswith('a'
songs_chart_df2[songs_chart_df2["Chart"].str.startswith('US')]["Chart"].value_counts()

US Billboard Hot 100                             167
US Hot R&B/Hip-Hop Songs (Billboard)              27
US Adult Contemporary (Billboard)                 20
US Hot Dance Club Songs (Billboard)               16
US Billboard Adult Contemporary                   11
US Billboard Hot Adult Contemporary Tracks         9
US Billboard Mainstream Rock Tracks                9
US Billboard Hot R&B/Hip-Hop Songs                 9
US Mainstream Top 40 (Billboard)                   8
US Billboard Modern Rock Tracks                    7
US Adult Top 40 (Billboard)                        6
US Hot Country Songs (Billboard)                   6
US Billboard Top 40 Mainstream                     5
US Rhythmic Top 40 (Billboard)                     4
US Billboard Adult Top 40                          4
US Billboard Triple A                              3
US Billboard Hot Modern Rock Tracks                3
US Dance/Club Play                                 3
US Dance Club Songs (Billboard)               

In [938]:
grossfiles = [(x,str(x)+'DomesticGross.csv') for x in range(1980,2014)]

print(grossfiles)

[(1980, '1980DomesticGross.csv'), (1981, '1981DomesticGross.csv'), (1982, '1982DomesticGross.csv'), (1983, '1983DomesticGross.csv'), (1984, '1984DomesticGross.csv'), (1985, '1985DomesticGross.csv'), (1986, '1986DomesticGross.csv'), (1987, '1987DomesticGross.csv'), (1988, '1988DomesticGross.csv'), (1989, '1989DomesticGross.csv'), (1990, '1990DomesticGross.csv'), (1991, '1991DomesticGross.csv'), (1992, '1992DomesticGross.csv'), (1993, '1993DomesticGross.csv'), (1994, '1994DomesticGross.csv'), (1995, '1995DomesticGross.csv'), (1996, '1996DomesticGross.csv'), (1997, '1997DomesticGross.csv'), (1998, '1998DomesticGross.csv'), (1999, '1999DomesticGross.csv'), (2000, '2000DomesticGross.csv'), (2001, '2001DomesticGross.csv'), (2002, '2002DomesticGross.csv'), (2003, '2003DomesticGross.csv'), (2004, '2004DomesticGross.csv'), (2005, '2005DomesticGross.csv'), (2006, '2006DomesticGross.csv'), (2007, '2007DomesticGross.csv'), (2008, '2008DomesticGross.csv'), (2009, '2009DomesticGross.csv'), (2010, '2

In [1394]:
#1983DomesticGross.csv


allGross_df = pd.DataFrame()

for year,file in grossfiles:
    #print(file)
    gross_df = pd.read_csv(file)
    gross_df["Year"] = year
    #print(gross_df.head(10))
    allGross_df = pd.concat([allGross_df,gross_df])
    
        

In [1521]:
allGross_df[allGross_df["Year"] == 1981]

,Close Date,Movie Title,Open Date,Opening Gross,Opening Theaters,Studio,Total Gross,Total Theaters,Year
0,NaN,The Cannonball Run,6/19,"$11,765,654","1,673",Fox,"$72,179,579","1,692",1981
1,NaN,"Fort Apache, The Bronx",2/6,NaN,NaN,Fox,"$29,200,000",NaN,1981
2,NaN,Sharky's Machine,12/18,"$3,022,041","1,388",WB,"$35,610,100","1,388",1981
3,NaN,The Devil and Max Devlin,2/6,NaN,NaN,BV,"$16,000,000",NaN,1981
4,NaN,The Postman Always Rings Twice,3/20,NaN,NaN,Par.,"$12,376,625",NaN,1981
5,NaN,Sphinx,2/13,"$439,564",214,WB,"$2,022,771",NaN,1981
6,NaN,Arthur,7/17,"$2,719,534",701,WB,"$95,461,682",701,1981
7,NaN,Raiders of the Lost Ark,6/12,"$8,305,823","1,078",Par.,"$212,222,025","1,078",1981
8,NaN,Rollover,12/11,"$2,260,689",755,WB,"$10,851,261",755,1981
9,NaN,The Fun House,3/13,"$2,765,456",814,Uni.,"$7,886,857",NaN,1981


In [961]:
allGross_df.head(10)

,Close Date,Movie Title,Open Date,Opening Gross,Opening Theaters,Studio,Total Gross,Total Theaters,Year
0,NaN,Saturn 3,2/15,NaN,NaN,AFD,"$9,000,000",NaN,1980
1,NaN,Little Miss Marker,3/21,"$853,173",472,Uni.,"$6,321,392",NaN,1980
2,NaN,The Empire Strikes Back,5/21,"$4,910,483",126,Fox,"$209,398,025","1,278",1980
3,NaN,Rough Cut,6/20,NaN,NaN,Par.,"$16,656,125",NaN,1980
4,NaN,The Nude Bomb,5/9,"$3,565,066",793,Uni.,"$14,662,035",NaN,1980
5,NaN,Urban Cowboy,6/6,NaN,NaN,Par.,"$46,918,287",NaN,1980
6,NaN,Popeye,12/12,"$6,310,520",901,Par.,"$49,823,037",901,1980
7,NaN,Used Cars,7/18,NaN,NaN,Col.,"$11,715,321",NaN,1980
8,NaN,Willie & Phil,-,NaN,NaN,Fox,"$4,400,000",NaN,1980
9,NaN,Any Which Way You Can,12/17,"$8,024,663","1,541",WB,"$70,687,344",NaN,1980


In [1478]:
#create dataframe for each movie and append to one

movies = []

songs = []

for title, d in singlesDict.items():
    movies.append(title)
    #print(d)
    temp_df = pd.DataFrame(d)
    temp_df.columns = ['Song Title','Wiki URL','Single Release Date']
    songs.append(temp_df)       
    #songs.append(pd.DataFrame.from_dict(d, orient='index'))
    
songs_df = pd.concat(songs, keys=movies)

In [1479]:
songs_df = songs_df.reset_index()

In [1480]:
songs_df.rename(columns = {'level_0':'Movie Title'}, inplace = True)
songs_df = songs_df.drop(axis=1, labels = ['level_1'])

In [1481]:
songs_df["Movie Title"].unique().size

207

In [1482]:
len(allGross_df)

3300

In [1535]:
songs_chart_df2[songs_chart_df2["Chart"]=='US Billboard Hot 100']

150

In [1627]:
songs_chart_us_df = songs_chart_df2[songs_chart_df2["Chart"].str.startswith('US')]

Create dummy variables to hold when the single charted

In [1628]:
songs_df["ChartBill100"] = songs_df["Song Title"].isin(songs_chart_us_df[songs_chart_us_df["Chart"] == 'US Billboard Hot 100']["Song Title"])

In [1629]:
#US Hot Dance Club Songs (Billboard)
songs_df["ChartHotDance"] = songs_df["Song Title"].isin(songs_chart_us_df[songs_chart_us_df["Chart"] == 'US Hot Dance Club Songs (Billboard)']["Song Title"])

In [1630]:
#US Hot R&B/Hip-Hop Songs (Billboard)
#or (songs_chart_us_df["Chart"] == 'US Hot R&B/Hip-Hop Songs')
songs_chart_hip_hop_df = songs_chart_us_df[songs_chart_us_df["Chart"].str.contains('US Hot R&B/Hip-Hop Songs')]
#songs_chart_hip_hop_df.size
songs_df["ChartHipHop"] = songs_df["Song Title"].isin(songs_chart_hip_hop_df["Song Title"])

In [1631]:
#df[~df.A.str.contains("Hello|World")]
songs_chart_us_other_df = songs_chart_us_df[~songs_chart_us_df["Chart"].str.contains('US Hot R&B/Hip-Hop Songs|US Hot Dance Club Songs (Billboard)|US Billboard Hot 100')]
#songs_chart_us_other_df.size
songs_df["ChartOther"] = songs_df["Song Title"].isin(songs_chart_us_other_df["Song Title"])

/Users/andrewvlahutin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app


In [1632]:
songs_df.columns

Index(['Movie Title', 'Song Title', 'Wiki URL', 'Single Release Date',
       'ChartBill100', 'ChartHotDance', 'ChartHipHop', 'ChartOther'],
      dtype='object')

songs_chart_us_pivot_df = pd.pivot_table(songs_chart_us_df,index=["Song Title"],columns='Chart',values='Chart Rank').reset_index()


In [1633]:
songs_df.head()

,Movie Title,Song Title,Wiki URL,Single Release Date,ChartBill100,ChartHotDance,ChartHipHop,ChartOther
0,Spirit: Stallion of the Cimarron,Here I Am,/wiki/Here_I_Am_(Bryan_Adams_song),"June 11, 2002",False,False,False,True
1,Toy Story 2,When She Loved Me,/wiki/When_She_Loved_Me,"November 24, 1999",False,False,False,False
2,Quantum of Solace,Another Way to Die,/wiki/Another_Way_to_Die,"September 30, 2008 (US), October 20, 2008",False,False,False,False
3,House Party 2,Ain't Gonna Hurt Nobody,,,False,False,False,False
4,House Party 2,House Party (I Don't Know What You Come to Do),,,False,False,False,False


songs_chart_us_pivot_df.size

#songs_df.merge(songs_chart_df,on='Song Title',how='left')
songs_chart_merge_df = pd.merge(left=songs_df,right=songs_chart_us_pivot_df,how='left')

In [1298]:
songs_chart_merge_df.head()

,Movie Title,Song Title,Wiki URL,Single Release Date,US Adult Contemporary (Billboard),US Adult Top 40 (Billboard),US Alternative Songs (Billboard),US Billboard Adult Contemporary,US Billboard Adult Top 40 Tracks,US Billboard European Hot 100 Singles,US Billboard Hot 100,US Billboard Hot Adult Contemporary Tracks,US Billboard Hot R&B/Hip-Hop Singles & Tracks,US Billboard Hot Rap Singles,US Billboard Mainstream Rock Tracks,US Billboard Mainstream Top 40,US Billboard Modern Rock Tracks,US Billboard Pop 100,US Billboard Top 40 Mainstream,US Bubbling Under Hot 100 Singles (Billboard),US Cashbox Top 100,US Hot 125,US Hot Country Songs (Billboard),US Hot Dance Club Songs (Billboard),US Hot R&B/Hip-Hop Songs (Billboard),US Mainstream Top 40 (Billboard),US Radio Songs (Billboard),US Rhythmic Top 40 (Billboard),US Rock Digital Songs
0,Kick-Ass,Kick Ass (We Are Young),/wiki/Kick_Ass_(We_Are_Young),2 May 2010 (UK),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spirit: Stallion of the Cimarron,Here I Am,/wiki/Here_I_Am_(Bryan_Adams_song),"June 11, 2002",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mulan,Reflection,/wiki/Reflection_(song),"June 2, 1998",19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Toy Story 2,When She Loved Me,/wiki/When_She_Loved_Me,"November 24, 1999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A Cinderella Story,Our Lips Are Sealed,/wiki/Our_Lips_Are_Sealed#Hilary_and_Haylie_Du...,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1278]:
songs_movies_charts_df = pd.merge(left=allGross_df,right=songs_chart_merge_df,how='left')

In [1283]:
songs_movies_charts_df.size

127613

In [1294]:
songs_movies_charts_df.head(10)

,Close Date,Movie Title,Open Date,Opening Gross,Opening Theaters,Studio,Total Gross,Total Theaters,Year,Song Title,Wiki URL,Single Release Date,US Adult Contemporary (Billboard),US Adult Top 40 (Billboard),US Alternative Songs (Billboard),US Billboard Adult Contemporary,US Billboard Adult Top 40 Tracks,US Billboard European Hot 100 Singles,US Billboard Hot 100,US Billboard Hot Adult Contemporary Tracks,US Billboard Hot R&B/Hip-Hop Singles & Tracks,US Billboard Hot Rap Singles,US Billboard Mainstream Rock Tracks,US Billboard Mainstream Top 40,US Billboard Modern Rock Tracks,US Billboard Pop 100,US Billboard Top 40 Mainstream,US Bubbling Under Hot 100 Singles (Billboard),US Cashbox Top 100,US Hot 125,US Hot Country Songs (Billboard),US Hot Dance Club Songs (Billboard),US Hot R&B/Hip-Hop Songs (Billboard),US Mainstream Top 40 (Billboard),US Radio Songs (Billboard),US Rhythmic Top 40 (Billboard),US Rock Digital Songs
0,NaN,Saturn 3,2/15,NaN,NaN,AFD,"$9,000,000",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Little Miss Marker,3/21,"$853,173",472,Uni.,"$6,321,392",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,The Empire Strikes Back,5/21,"$4,910,483",126,Fox,"$209,398,025","1,278",1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Rough Cut,6/20,NaN,NaN,Par.,"$16,656,125",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,The Nude Bomb,5/9,"$3,565,066",793,Uni.,"$14,662,035",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Urban Cowboy,6/6,NaN,NaN,Par.,"$46,918,287",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Popeye,12/12,"$6,310,520",901,Par.,"$49,823,037",901,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Used Cars,7/18,NaN,NaN,Col.,"$11,715,321",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Willie & Phil,-,NaN,NaN,Fox,"$4,400,000",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,Any Which Way You Can,12/17,"$8,024,663","1,541",WB,"$70,687,344",NaN,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1013]:
songs_movies_charts_df.columns

Index(['Close Date', 'Movie Title', 'Open Date', 'Opening Gross',
       'Opening Theaters', 'Studio', 'Total Gross', 'Total Theaters', 'Year',
       'Song Title', 'Wiki URL', 'Single Release Date',
       'US Adult Contemporary (Billboard)', 'US Adult Top 40 (Billboard)',
       'US Alternative Songs (Billboard)', 'US Billboard Adult Contemporary',
       'US Billboard Adult Top 40 Tracks',
       'US Billboard European Hot 100 Singles', 'US Billboard Hot 100',
       'US Billboard Hot Adult Contemporary Tracks',
       'US Billboard Hot R&B/Hip-Hop Singles & Tracks',
       'US Billboard Hot Rap Singles', 'US Billboard Mainstream Rock Tracks',
       'US Billboard Mainstream Top 40', 'US Billboard Modern Rock Tracks',
       'US Billboard Pop 100', 'US Billboard Top 40 Mainstream',
       'US Bubbling Under Hot 100 Singles (Billboard)', 'US Cashbox Top 100',
       'US Hot 125', 'US Hot Country Songs (Billboard)',
       'US Hot Dance Club Songs (Billboard)',
       'US Hot R&B/Hip-

In [1393]:
len(allGross_df)

3300

In [1634]:
model_df = allGross_df.copy(deep=True)

In [1635]:
model_df["HasSingle"] = model_df['Movie Title'].isin(songs_df['Movie Title'])

In [1636]:
#df.query('A in [1,3] and B in [4,7,12]')
#model_df.query(r'"Movie Title" in songs_df["Movie Title"] and True in songs_df["ChartBill100"]')

songs_hot_100_df = songs_df[songs_df["ChartBill100"]==True]
model_df["ChartHot100"] = model_df['Movie Title'].isin(songs_hot_100_df['Movie Title'])

In [1637]:
songs_hot_100_df = songs_df[songs_df["ChartHotDance"]==True]
model_df["ChartHotDance"] = model_df['Movie Title'].isin(songs_hot_100_df['Movie Title'])

In [1638]:
songs_hot_100_df = songs_df[songs_df["ChartHipHop"]==True]
model_df["ChartHipHop"] = model_df['Movie Title'].isin(songs_hot_100_df['Movie Title'])

In [1639]:
songs_hot_100_df = songs_df[songs_df["ChartOther"]==True]
model_df["ChartOther"] = model_df['Movie Title'].isin(songs_hot_100_df['Movie Title'])

In [1640]:
model_df.columns

Index(['Close Date', 'Movie Title', 'Open Date', 'Opening Gross',
       'Opening Theaters', 'Studio', 'Total Gross', 'Total Theaters', 'Year',
       'HasSingle', 'ChartHot100', 'ChartHotDance', 'ChartHipHop',
       'ChartOther'],
      dtype='object')

In [1500]:
model_df.head(2)

,Total Gross,HasSingle,ChartHot100,ChartHotDance,ChartHipHop,ChartOther
0,9000000.0,False,False,False,False,False
1,6321392.0,False,False,False,False,False


In [1642]:
model_df = model_df.drop(axis=1,labels=['Close Date','Open Date', 'Opening Gross','Movie Title','Opening Theaters','Studio','Total Theaters','Year'])

In [1643]:
model_df['Total Gross'] = model_df['Total Gross'].replace('[\$,]', '', regex=True).astype(float)

__Now Fit the Model__

import pickle

with open('songs_movies_charts_model_minusdt_df.pkl', 'wb') as picklefile:
    pickle.dump(songs_movies_charts_model_minusdt_df, picklefile)

#songs_chart_merge_df

with open('songs_chart_merge_df.pkl', 'wb') as picklefile:
    pickle.dump(songs_chart_merge_df, picklefile)

In [1641]:
with open('model_df.pkl', 'wb') as picklefile:
    pickle.dump(model_df, picklefile)

In [1644]:
len(model_df)

3300

In [1645]:
model_df.head(10)

,Total Gross,HasSingle,ChartHot100,ChartHotDance,ChartHipHop,ChartOther
0,9000000.0,False,False,False,False,False
1,6321392.0,False,False,False,False,False
2,209398025.0,False,False,False,False,False
3,16656125.0,False,False,False,False,False
4,14662035.0,False,False,False,False,False
5,46918287.0,False,False,False,False,False
6,49823037.0,False,False,False,False,False
7,11715321.0,False,False,False,False,False
8,4400000.0,False,False,False,False,False
9,70687344.0,False,False,False,False,False


In [1503]:
model_df["HasSingle"].value_counts()

False    3098
True      202
Name: HasSingle, dtype: int64

In [1646]:
sample_model_single_df = model_df[model_df["HasSingle"]==True]

In [1647]:
sample_model_x_single_df = model_df[~model_df["HasSingle"]==True]

In [1648]:
print(len(sample_model_single_df))
print(len(sample_model_x_single_df))

202
3098


In [1694]:
sample_model_x_single_df2 = sample_model_x_single_df.sample(202)

In [1695]:
print(len(sample_model_x_single_df2))

202


In [1696]:
oversample_model_df = pd.concat([sample_model_single_df,sample_model_x_single_df2])

In [1697]:
len(oversample_model_df)

404

In [1698]:
X,y = oversample_model_df.drop('Total Gross', axis=1),oversample_model_df['Total Gross']

In [1699]:
from sklearn import linear_model


models = {}
models['lin_reg'] = linear_model.LinearRegression()
models['ridge'] = linear_model.Ridge()
models['lasso'] = linear_model.Lasso(alpha=.001)
models['elasticnet'] = linear_model.ElasticNet(alpha=.002)

In [1700]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
                                        X, y, test_size=0.3)
    
#X_train, X_test, y_train, y_test =   cross_validation.StratifiedShuffleSplit(X, y, test_size=0.3)

In [1701]:
for name,model in models.items():
    model.fit(X_train,y_train)
    print('Model: '+name)
    print("Score: " + str(model.score(X_test,y_test)))
    sorted_features = sorted(zip(X,model.coef_), key=lambda tup: abs(tup[1]), reverse=True)
    for feature in sorted_features:
        print(feature)
        
    print("")

Model: lasso
Score: 0.0591866616567
('HasSingle', 26029866.746479217)
('ChartHotDance', 12546218.595457461)
('ChartHipHop', -11318477.479630077)
('ChartOther', 8956804.7999361511)
('ChartHot100', -6468696.8161089672)

Model: lin_reg
Score: 0.0591866615968
('HasSingle', 26029866.757858325)
('ChartHotDance', 12546218.927936705)
('ChartHipHop', -11318477.53114832)
('ChartOther', 8956804.8028676156)
('ChartHot100', -6468696.8273773389)

Model: elasticnet
Score: 0.0596549848831
('HasSingle', 25818067.49252297)
('ChartHipHop', -10774960.033179557)
('ChartHotDance', 9364213.4192069154)
('ChartOther', 8976022.9622021616)
('ChartHot100', -6328203.9530124702)

Model: ridge
Score: 0.0599585707733
('HasSingle', 25322609.938477904)
('ChartHipHop', -9830063.2853477504)
('ChartOther', 8940938.4826182071)
('ChartHot100', -5973775.7180805011)
('ChartHotDance', 5556282.0893219551)



In [1681]:
import statsmodels.api as sm

predictors = sm.add_constant(X)

In [1682]:
model = sm.OLS(y, X)
results = model.fit()

In [1683]:
results.params

HasSingle        8.720029e+07
ChartHot100      1.487006e+06
ChartHotDance   -1.609111e+07
ChartHipHop     -2.117392e+07
ChartOther       1.095270e+07
dtype: float64

In [1684]:
predictors.head()

,const,HasSingle,ChartHot100,ChartHotDance,ChartHipHop,ChartOther
13,1,True,True,False,False,False
18,1,True,False,False,False,False
44,1,True,False,False,False,False
66,1,True,True,False,False,False
68,1,True,False,False,False,False


In [1685]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            Total Gross   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.338
Method:                 Least Squares   F-statistic:                     42.28
Date:                Thu, 06 Oct 2016   Prob (F-statistic):           6.41e-35
Time:                        19:40:29   Log-Likelihood:                -7959.7
No. Observations:                 404   AIC:                         1.593e+04
Df Residuals:                     399   BIC:                         1.595e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
HasSingle       8.72e+07   9.75e+06      8.946